In [22]:
import dp

In [23]:
threshold = 0.90

In [24]:
# Cosine similarity matching function for traffic_countries & forest_countries
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(dp.countries_traffic)+list(dp.countries_forest))

# Matching process
matches_tf = []
for loc in dp.countries_traffic:
    for country in dp.countries_forest:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Forest",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Forest matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Forest matches (sorted by similarity):
      Source  Target                Value_A                Value_B  Similarity
95   Traffic  Forest  sao tome and principe  sao tome and principe         1.0
0    Traffic  Forest             san marino             san marino         1.0
60   Traffic  Forest              sri lanka              sri lanka         1.0
109  Traffic  Forest           burkina faso           burkina faso         1.0
125  Traffic  Forest           sierra leone           sierra leone         1.0
..       ...     ...                    ...                    ...         ...
51   Traffic  Forest                 brazil                 brazil         1.0
52   Traffic  Forest                 mexico                 mexico         1.0
53   Traffic  Forest                georgia                georgia         1.0
54   Traffic  Forest                 israel                 israel         1.0
150  Traffic  Forest                somalia                somalia         1.0

[151

In [25]:
# Cosine similarity matching function for # traffic_countries & countries_air_country
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(dp.countries_traffic)+list(dp.countries_air_country))

# Matching process
matches_tf = []
for loc in dp.countries_traffic:
    for country in dp.countries_air_country:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Air_Country",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Air_Country matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Air_Country matches (sorted by similarity):
      Source       Target           Value_A           Value_B  Similarity
0    Traffic  Air_Country        san marino        san marino         1.0
22   Traffic  Air_Country    czech republic    czech republic         1.0
162  Traffic  Air_Country  papua new guinea  papua new guinea         1.0
143  Traffic  Air_Country      sierra leone      sierra leone         1.0
124  Traffic  Air_Country      burkina faso      burkina faso         1.0
..       ...          ...               ...               ...         ...
69   Traffic  Air_Country            russia            russia         1.0
70   Traffic  Air_Country            serbia            serbia         1.0
71   Traffic  Air_Country            turkey            turkey         1.0
72   Traffic  Air_Country          mongolia          mongolia         1.0
85   Traffic  Air_Country            kosovo            kosovo         1.0

[170 rows x 5 columns]


In [26]:
# Cosine similarity matching function for # traffic_countries & countries_air_city
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(dp.countries_traffic)+list(dp.countries_air_city))

# Matching process
matches_tf = []
for loc in dp.countries_traffic:
    for country in dp.countries_air_city:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Air_City",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Air_City matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Air_City matches (sorted by similarity):
     Source    Target         Value_A         Value_B  Similarity
57  Traffic  Air_City    saudi arabia    saudi arabia         1.0
63  Traffic  Air_City     el salvador     el salvador         1.0
34  Traffic  Air_City      costa rica      costa rica         1.0
0   Traffic  Air_City         andorra         andorra         1.0
51  Traffic  Air_City         albania         albania         1.0
..      ...       ...             ...             ...         ...
25  Traffic  Air_City         austria         austria         1.0
24  Traffic  Air_City        slovenia        slovenia         1.0
23  Traffic  Air_City         belgium         belgium         1.0
22  Traffic  Air_City        slovakia        slovakia         1.0
20  Traffic  Air_City  united kingdom  united kingdom         1.0

[80 rows x 5 columns]


In [27]:
# Cosine similarity matching function for # traffic_countries & countries_air_weather
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(dp.countries_traffic)+list(dp.countries_air_weather))

# Matching process
matches_tf = []
for loc in dp.countries_traffic:
    for country in dp.countries_air_weather:
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Source": "Traffic",
                "Target": "Air_Weather",
                "Value_A": loc,
                "Value_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Air_Weather matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Air_Weather matches (sorted by similarity):
      Source       Target       Value_A       Value_B  Similarity
0    Traffic  Air_Weather    san marino    san marino         1.0
130  Traffic  Air_Weather  sierra leone  sierra leone         1.0
83   Traffic  Air_Weather  south africa  south africa         1.0
84   Traffic  Air_Weather   saint lucia   saint lucia         1.0
97   Traffic  Air_Weather   el salvador   el salvador         1.0
..       ...          ...           ...           ...         ...
55   Traffic  Air_Weather        mexico        mexico         1.0
56   Traffic  Air_Weather       georgia       georgia         1.0
57   Traffic  Air_Weather        israel        israel         1.0
58   Traffic  Air_Weather      barbados      barbados         1.0
155  Traffic  Air_Weather   north korea   north korea         1.0

[156 rows x 5 columns]


In [28]:
# Cosine similarity matching function for # city_forest & city_air_city
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(dp.city_forest)+list(dp.city_air_city))

# Matching process
matches_tf = []
for loc in dp.city_forest:
    for city in dp.city_air_city:
        is_match, similarity = cosine_match(vectorizer, loc, city)
        if is_match:
            matches_tf.append({
                "Source": "City_Forest",
                "Target": "City_Air_City",
                "Value_A": loc,
                "Value_B": city,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("City_Forest - City_Air_City matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No City_Air_City matches found")

AttributeError: module 'dp' has no attribute 'city_forest'

In [ ]:
# Cosine similarity matching function for # city_forest & city_air_weather
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(dp.city_forest)+list(dp.city_air_weather))

# Matching process
matches_tf = []
for loc in dp.city_forest:
    for city in dp.city_air_weather:
        is_match, similarity = cosine_match(vectorizer, loc, city)
        if is_match:
            matches_tf.append({
                "Source": "City_Forest",
                "Target": "City_Air_Weather",
                "Value_A": loc,
                "Value_B": city,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("City_Forest - City_Air_Weather matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No City_Air_Weather matches found")

City_Forest - City_Air_Weather matches (sorted by similarity):
          Source            Target         Value_A         Value_B  Similarity
45   City_Forest  City_Air_Weather   santo domingo   santo domingo         1.0
52   City_Forest  City_Air_Weather     addis ababa     addis ababa         1.0
74   City_Forest  City_Air_Weather       new delhi       new delhi         1.0
101  City_Forest  City_Air_Weather     mexico city     mexico city         1.0
67   City_Forest  City_Air_Weather  guatemala city  guatemala city         1.0
..           ...               ...             ...             ...         ...
60   City_Forest  City_Air_Weather           accra           accra         1.0
61   City_Forest  City_Air_Weather         conakry         conakry         1.0
62   City_Forest  City_Air_Weather          banjul          banjul         1.0
63   City_Forest  City_Air_Weather          bissau          bissau         1.0
173  City_Forest  City_Air_Weather          harare          harare  